In [ ]:
%matplotlib inline
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

## Load the data

In [ ]:
stops = gpd.read_file("data/metlink-stops.gpkg").to_crs(2193)
routes = gpd.read_file("data/metlink-routes.gpkg").to_crs(2193)
sa2 = gpd.read_file("data/sa2-wellington.gpkg")

# Unary GIS operations
As a general comment on all spatial operations in `geopandas`, they almost always return a `GeoSeries` _not_ a `GeoDataFrame`. Usually that means you have to do something like

```python
gdf.geometry = gdf.geometry.<some_function()>
```

if you want to change the geometry of a dataset. Something like

```python
gdf = gdf.<some_function>
```

will most likely change your `GeoDataFrame` into a `GeoSeries` and in the process discard all the data. It's not difficult to remember this, but it can sometimes seems a little bit roundabout. An additional thing to note is that most methods can be applied either to `GeoDataFrame` or to `GeoSeries` objects, so these two lines will have the same effect:

```python
gdf.geometry = gdf.geometry.<some_method()>
gdf.geometry = gdf.<some_method()>
```

The important part is that the results are almost always a `GeoSeries` (yes... I'm repeating myself, but it's important!).

A side-effect of this is that making a new `GeoDataFrame` is a two step process:

```python
new_gdf = gdf.copy()
new_gdf.geometry = gdf.<some_method()>
```

## Buffering
The buffer operation is straightforward. Key things to remember:
- Distances are in the units of the CRS (and only projected coordinate systems make sense for buffering)
- Negative distances are OK for polygons, but will cause either errors or things to disappear if set too large
- Options include `cap_style`, `join_style`, `mitre_limit`, and `single_sided` are available and [documented here](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.buffer.html)

In [ ]:
stops_buffer = stops.copy()
stops_buffer.geometry = stops.buffer(500)
m = stops_buffer.explore(style_kwds = {"color": "#ff000010"}, 
                         tiles = "CartoDB.Positron")
stops.explore(m = m, marker_kwds = {"color": "k"})

Buffering a dataset based on some numeric attribute in the data is straightforward. Below I add a new `dist` attribute to the stops and give it a series of random numbers, just to demonstrate things. Of note here is a different way to add a new variable to a table, the `assign()` method, which creates a new table with the added column calculated as indicated.

In [ ]:
stops_buffer_2 = stops.assign(dist = range(100, 100 + stops.shape[0]))
stops_buffer_2.geometry = stops_buffer_2.geometry.buffer(stops_buffer_2.dist)
m = stops_buffer_2.explore(style_kwds = {"color": "#ff000010"}, 
                           tiles = "CartoDB.Positron")
stops.explore(m = m, marker_kwds = {"color": "k"})

## Bounding boxes
`bounds` gives you the limiting coordinates of each geometry in a `GeoSeries`; `total_bounds` gives you the limiting coordinates of the whole dataset. These can be useful, but perhaps more interesting is `envelope` which gives a bounding rectangle for each geometry in a `GeoSeries`.

In [ ]:
routes_bb = routes.copy()
routes_bb.geometry = routes.envelope
m = routes_bb.explore(style_kwds = {"color": "#ff000020"}, 
                      tiles = "CartoDB.Positron")
routes.explore(m = m, style_kwds = {"color": "black"})

## Other representative shapes
In the same vein as bounding boxes are `minimum_rotated_rectangle()`, `minimum_bounding_circle()`, and `convex_hull`. Note that the first two are methods while the last (like `envelope`) is an attribute.

In [ ]:
routes.minimum_rotated_rectangle().explore(
    style_kwds = {"color": "#ff000020"}, tiles = "CartoDB.Positron")

In [ ]:
routes.minimum_bounding_circle().explore(
    style_kwds = {"color": "#ff000020"}, tiles = "CartoDB.Positron")

In [ ]:
routes.convex_hull.explore(
    style_kwds = {"color": "#ff000020"}, tiles = "CartoDB.Positron")

## Representative points
`centroid` and `representative_point()` which we've seen before when labelling a map, are easily determined.

In [ ]:
m = sa2.explore(tooltip = False, tiles = "CartoDB.Positron")
sa2.centroid.explore(m = m, style_kwds = {"color": "black"})
sa2.representative_point().explore(m = m, style_kwds = {"color": "red"})

## Other spatial operations
Worth exploring are
- [`boundary`](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.boundary.html) which returns the outer boundaries of the input geometries - the exterior of polygons as lines, the end points of lines as points
- [`concave_hull()`](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.concave_hull.html) is another possible summary shape for complex geometries that is less dramatically simplifying than `convex_hull` (requires geos 3.11)
- [`maximum_inscribed_circle()`](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.maximum_inscribed_circle.html) returns the largest circles that will fit inside each polygon in a `GeoSeries`
- [`segmentize()`](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.segmentize.html) adds points along the lines in a geometry to 'densify' them
- [`simplify()`](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.simplify.html) and [`simplify_coverage()`](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.simplify_coverage.html) can be used to reduce the complexity of shapes in a way that is suitable for map generalisation

... and many more. Consult [the `GeoSeries` documentation](https://geopandas.org/en/stable/docs/reference/geoseries.html#constructive-methods-and-attributes)!